<a href="https://colab.research.google.com/github/hftscan/nifty50/blob/main/HFtscan_nifty50_past_2_year_(10000_strategy_).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install backtrader yfinance pandas matplotlib

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 53.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 157.6 MB/s eta 0:00:00
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15636 sha256=8a0bb03fb0d2e2d140fc6ccb60decc490b9bd8c24d8de52beb632d21bb3a5e4c
  Stored in directory: /root/.cache/pip/wheels/cc/bd/6f/664d62c99327abeef7d86489e6631cbf45b56fbf7ef1d6ef00
  Created wheel for peewee: filename=peewee-3.18.3-cp312-cp312-linux_x86_64.whl size=936695 sha256=7fc9b671e758400e17fccd4358825048cb8838ba3a

In [6]:
import backtrader as bt
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from itertools import product
import warnings
warnings.filterwarnings('ignore')

# ==================== CONFIGURATION ====================
CONFIG = {
    'START_DATE': datetime.now() - timedelta(days=2*365),  # 2 years back
    'END_DATE': datetime.now(),
    'INITIAL_CASH': 100000,
    'COMMISSION': 0.001,  # 0.1%
    'TIMEFRAME': 'daily',  # Options: 'daily', '1h', '15m', '5m'
    'MAX_STRATEGIES_TO_TEST': 10000,  # Maximum combinations to test
    'TOP_N_RESULTS': 20,  # Show top N best strategies
}

# NIFTY 50 stocks (as of 2024)
NIFTY50_TICKERS = [
    'RELIANCE.NS', 'TCS.NS', 'HDFCBANK.NS', 'INFY.NS', 'ICICIBANK.NS',
    'HINDUNILVR.NS', 'ITC.NS', 'SBIN.NS', 'BHARTIARTL.NS', 'KOTAKBANK.NS',
    'LT.NS', 'AXISBANK.NS', 'ASIANPAINT.NS', 'MARUTI.NS', 'HCLTECH.NS',
    'WIPRO.NS', 'ULTRACEMCO.NS', 'TITAN.NS', 'BAJFINANCE.NS', 'NESTLEIND.NS',
    'SUNPHARMA.NS', 'TECHM.NS', 'ONGC.NS', 'NTPC.NS', 'TATAMOTORS.NS',
    'POWERGRID.NS', 'M&M.NS', 'TATASTEEL.NS', 'BAJAJFINSV.NS', 'ADANIPORTS.NS',
    'COALINDIA.NS', 'HINDALCO.NS', 'DIVISLAB.NS', 'HEROMOTOCO.NS', 'BRITANNIA.NS',
    'CIPLA.NS', 'EICHERMOT.NS', 'DRREDDY.NS', 'JSWSTEEL.NS', 'GRASIM.NS',
    'INDUSINDBK.NS', 'APOLLOHOSP.NS', 'BPCL.NS', 'TATACONSUM.NS', 'SHRIRAMFIN.NS',
    'ADANIENT.NS', 'SBILIFE.NS', 'HDFCLIFE.NS', 'BAJAJ-AUTO.NS', 'LTIM.NS'
]

# ==================== STRATEGY PARAMETER RANGES ====================
STRATEGY_PARAMS = {
    'MeanReversion': {
        'bb_period': [10, 20, 30, 50],
        'bb_devfactor': [1.5, 2, 2.5, 3],
    },
    'Momentum': {
        'fast_period': [5, 10, 15, 20],
        'slow_period': [30, 50, 100, 200],
    },
    'RSI': {
        'period': [7, 14, 21, 28],
        'oversold': [20, 25, 30, 35],
        'overbought': [65, 70, 75, 80],
    },
    'MACD': {
        'fast': [8, 12, 16],
        'slow': [21, 26, 30],
        'signal': [7, 9, 11],
    },
    'DualMA': {
        'fast': [5, 10, 20],
        'slow': [50, 100, 200],
    },
    'TripleMA': {
        'fast': [5, 10],
        'medium': [20, 50],
        'slow': [100, 200],
    },
    'Stochastic': {
        'period': [14, 21],
        'period_dfast': [3, 5],
        'oversold': [20, 30],
        'overbought': [70, 80],
    },
    'CCI': {
        'period': [14, 20, 30],
        'entry': [100, 150, 200],
    },
    'ADX': {
        'period': [14, 20],
        'threshold': [20, 25, 30],
    },
    'ATR': {
        'period': [14, 21],
        'multiplier': [2, 3, 4],
    },
}


# ==================== STRATEGIES ====================
class MeanReversionStrategy(bt.Strategy):
    params = (('bb_period', 20), ('bb_devfactor', 2))

    def __init__(self):
        self.bbands = bt.indicators.BollingerBands(
            self.data.close, period=self.params.bb_period, devfactor=self.params.bb_devfactor
        )
        self.order = None

    def next(self):
        if self.order:
            return
        if not self.position:
            if self.data.close[0] < self.bbands.lines.bot[0]:
                self.order = self.buy()
        else:
            if self.data.close[0] > self.bbands.lines.top[0]:
                self.order = self.sell()

    def notify_order(self, order):
        if order.status in [order.Completed]:
            self.order = None


class MomentumStrategy(bt.Strategy):
    params = (('fast_period', 10), ('slow_period', 50))

    def __init__(self):
        self.fast_ma = bt.indicators.SMA(self.data.close, period=self.params.fast_period)
        self.slow_ma = bt.indicators.SMA(self.data.close, period=self.params.slow_period)
        self.crossover = bt.indicators.CrossOver(self.fast_ma, self.slow_ma)
        self.order = None

    def next(self):
        if self.order:
            return
        if not self.position:
            if self.crossover > 0:
                self.order = self.buy()
        else:
            if self.crossover < 0:
                self.order = self.sell()

    def notify_order(self, order):
        if order.status in [order.Completed]:
            self.order = None


class RSIStrategy(bt.Strategy):
    params = (('period', 14), ('oversold', 30), ('overbought', 70))

    def __init__(self):
        self.rsi = bt.indicators.RSI(self.data.close, period=self.params.period)
        self.order = None

    def next(self):
        if self.order:
            return
        if not self.position:
            if self.rsi[0] < self.params.oversold:
                self.order = self.buy()
        else:
            if self.rsi[0] > self.params.overbought:
                self.order = self.sell()

    def notify_order(self, order):
        if order.status in [order.Completed]:
            self.order = None


class MACDStrategy(bt.Strategy):
    params = (('fast', 12), ('slow', 26), ('signal', 9))

    def __init__(self):
        self.macd = bt.indicators.MACD(
            self.data.close, period_me1=self.params.fast,
            period_me2=self.params.slow, period_signal=self.params.signal
        )
        self.crossover = bt.indicators.CrossOver(self.macd.macd, self.macd.signal)
        self.order = None

    def next(self):
        if self.order:
            return
        if not self.position:
            if self.crossover > 0:
                self.order = self.buy()
        else:
            if self.crossover < 0:
                self.order = self.sell()

    def notify_order(self, order):
        if order.status in [order.Completed]:
            self.order = None


class DualMAStrategy(bt.Strategy):
    params = (('fast', 10), ('slow', 50))

    def __init__(self):
        self.fast_ema = bt.indicators.EMA(self.data.close, period=self.params.fast)
        self.slow_ema = bt.indicators.EMA(self.data.close, period=self.params.slow)
        self.crossover = bt.indicators.CrossOver(self.fast_ema, self.slow_ema)
        self.order = None

    def next(self):
        if self.order:
            return
        if not self.position:
            if self.crossover > 0:
                self.order = self.buy()
        else:
            if self.crossover < 0:
                self.order = self.sell()

    def notify_order(self, order):
        if order.status in [order.Completed]:
            self.order = None


class TripleMAStrategy(bt.Strategy):
    params = (('fast', 5), ('medium', 20), ('slow', 100))

    def __init__(self):
        self.fast = bt.indicators.SMA(self.data.close, period=self.params.fast)
        self.medium = bt.indicators.SMA(self.data.close, period=self.params.medium)
        self.slow = bt.indicators.SMA(self.data.close, period=self.params.slow)
        self.order = None

    def next(self):
        if self.order:
            return
        if not self.position:
            if self.fast[0] > self.medium[0] > self.slow[0]:
                self.order = self.buy()
        else:
            if self.fast[0] < self.medium[0]:
                self.order = self.sell()

    def notify_order(self, order):
        if order.status in [order.Completed]:
            self.order = None


class StochasticStrategy(bt.Strategy):
    params = (('period', 14), ('period_dfast', 3), ('oversold', 20), ('overbought', 80))

    def __init__(self):
        self.stoch = bt.indicators.Stochastic(
            self.data, period=self.params.period, period_dfast=self.params.period_dfast
        )
        self.order = None

    def next(self):
        if self.order:
            return
        if not self.position:
            if self.stoch.percK[0] < self.params.oversold and self.stoch.percD[0] < self.params.oversold:
                self.order = self.buy()
        else:
            if self.stoch.percK[0] > self.params.overbought and self.stoch.percD[0] > self.params.overbought:
                self.order = self.sell()

    def notify_order(self, order):
        if order.status in [order.Completed]:
            self.order = None


class CCIStrategy(bt.Strategy):
    params = (('period', 20), ('entry', 100))

    def __init__(self):
        self.cci = bt.indicators.CCI(self.data, period=self.params.period)
        self.order = None

    def next(self):
        if self.order:
            return
        if not self.position:
            if self.cci[0] < -self.params.entry:
                self.order = self.buy()
        else:
            if self.cci[0] > self.params.entry:
                self.order = self.sell()

    def notify_order(self, order):
        if order.status in [order.Completed]:
            self.order = None


class ADXStrategy(bt.Strategy):
    params = (('period', 14), ('threshold', 25))

    def __init__(self):
        self.adx = bt.indicators.ADX(self.data, period=self.params.period)
        self.dmi_plus = self.adx.lines.plusDI
        self.dmi_minus = self.adx.lines.minusDI
        self.order = None

    def next(self):
        if self.order:
            return
        if not self.position:
            if self.adx[0] > self.params.threshold and self.dmi_plus[0] > self.dmi_minus[0]:
                self.order = self.buy()
        else:
            if self.dmi_minus[0] > self.dmi_plus[0]:
                self.order = self.sell()

    def notify_order(self, order):
        if order.status in [order.Completed]:
            self.order = None


class ATRStrategy(bt.Strategy):
    params = (('period', 14), ('multiplier', 3))

    def __init__(self):
        self.atr = bt.indicators.ATR(self.data, period=self.params.period)
        self.sma = bt.indicators.SMA(self.data.close, period=50)
        self.order = None

    def next(self):
        if self.order:
            return

        upper_band = self.sma[0] + (self.params.multiplier * self.atr[0])
        lower_band = self.sma[0] - (self.params.multiplier * self.atr[0])

        if not self.position:
            if self.data.close[0] < lower_band:
                self.order = self.buy()
        else:
            if self.data.close[0] > upper_band:
                self.order = self.sell()

    def notify_order(self, order):
        if order.status in [order.Completed]:
            self.order = None


# ==================== DATA FETCHING ====================
def get_data(ticker, start_date, end_date):
    """Download stock data from Yahoo Finance"""
    try:
        df = yf.download(ticker, start=start_date, end=end_date, auto_adjust=True, progress=False)

        if df.empty:
            return None

        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)

        df.columns = [str(col).lower() for col in df.columns]
        df.rename(columns={'adj close': 'close'}, inplace=True)

        # Make sure we have all required columns
        required_cols = ['open', 'high', 'low', 'close', 'volume']
        for col in required_cols:
            if col not in df.columns:
                return None

        # Return the dataframe, not the feed (we'll create feed in cerebro)
        return df
    except Exception as e:
        print(f"Error downloading {ticker}: {e}")
        return None


# ==================== OPTIMIZER ====================
def run_single_backtest(strategy_class, params, data_df, ticker, silent=True):
    """Run a single backtest and return results"""
    try:
        cerebro = bt.Cerebro()
        cerebro.addstrategy(strategy_class, **params)

        # Create data feed here inside cerebro context
        data = bt.feeds.PandasData(dataname=data_df)
        cerebro.adddata(data)

        cerebro.broker.setcash(CONFIG['INITIAL_CASH'])
        cerebro.broker.setcommission(commission=CONFIG['COMMISSION'])

        cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe', riskfreerate=0.06)
        cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
        cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
        cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trades')

        results = cerebro.run()
        strat = results[0]

        final_value = cerebro.broker.getvalue()
        pnl = final_value - CONFIG['INITIAL_CASH']
        pnl_pct = (pnl / CONFIG['INITIAL_CASH']) * 100

        sharpe = strat.analyzers.sharpe.get_analysis().get('sharperatio', None)
        drawdown = strat.analyzers.drawdown.get_analysis()
        max_dd = drawdown.max.drawdown if hasattr(drawdown, 'max') else 0

        trades = strat.analyzers.trades.get_analysis()
        total_trades = trades.total.closed if hasattr(trades.total, 'closed') else 0

        win_rate = 0
        if total_trades > 0 and hasattr(trades, 'won'):
            won = trades.won.total if hasattr(trades.won, 'total') else 0
            win_rate = (won / total_trades * 100) if total_trades > 0 else 0

        return {
            'ticker': ticker,
            'strategy': strategy_class.__name__,
            'params': str(params),
            'final_value': final_value,
            'pnl': pnl,
            'return_pct': pnl_pct,
            'sharpe': sharpe if sharpe else 0,
            'max_drawdown': max_dd,
            'total_trades': total_trades,
            'win_rate': win_rate,
        }
    except Exception as e:
        if not silent:
            print(f"Error in backtest: {e}")
        return None


def optimize_strategies():
    """Test all strategy combinations and find best performers"""
    print("="*80)
    print("NIFTY 50 STRATEGY OPTIMIZER")
    print("="*80)
    print(f"Configuration:")
    print(f"  - Period: {CONFIG['START_DATE'].strftime('%Y-%m-%d')} to {CONFIG['END_DATE'].strftime('%Y-%m-%d')}")
    print(f"  - Initial Capital: ₹{CONFIG['INITIAL_CASH']:,}")
    print(f"  - Commission: {CONFIG['COMMISSION']*100}%")
    print(f"  - Max Strategies to Test: {CONFIG['MAX_STRATEGIES_TO_TEST']}")
    print("="*80)

    # Download all data first
    print("\nDownloading NIFTY 50 stock data...")
    stock_data = {}
    for ticker in NIFTY50_TICKERS:
        data_df = get_data(ticker, CONFIG['START_DATE'], CONFIG['END_DATE'])
        if data_df is not None and not data_df.empty:
            stock_data[ticker] = data_df
            print(f"✓ {ticker}")
        else:
            print(f"✗ {ticker} - Failed")

    print(f"\nSuccessfully loaded {len(stock_data)} stocks")

    # Generate all strategy combinations
    print("\nGenerating strategy combinations...")
    all_results = []
    test_count = 0

    strategy_classes = {
        'MeanReversion': MeanReversionStrategy,
        'Momentum': MomentumStrategy,
        'RSI': RSIStrategy,
        'MACD': MACDStrategy,
        'DualMA': DualMAStrategy,
        'TripleMA': TripleMAStrategy,
        'Stochastic': StochasticStrategy,
        'CCI': CCIStrategy,
        'ADX': ADXStrategy,
        'ATR': ATRStrategy,
    }

    for strategy_name, strategy_class in strategy_classes.items():
        param_names = list(STRATEGY_PARAMS[strategy_name].keys())
        param_values = [STRATEGY_PARAMS[strategy_name][p] for p in param_names]

        for param_combo in product(*param_values):
            params = dict(zip(param_names, param_combo))

            for ticker, data_df in stock_data.items():
                if test_count >= CONFIG['MAX_STRATEGIES_TO_TEST']:
                    break

                result = run_single_backtest(strategy_class, params, data_df, ticker)
                if result:
                    all_results.append(result)
                    test_count += 1

                    if test_count % 100 == 0:
                        print(f"Progress: {test_count}/{CONFIG['MAX_STRATEGIES_TO_TEST']} tests completed...")

            if test_count >= CONFIG['MAX_STRATEGIES_TO_TEST']:
                break

        if test_count >= CONFIG['MAX_STRATEGIES_TO_TEST']:
            break

    print(f"\nCompleted {test_count} backtests!")

    # Convert to DataFrame and sort
    df_results = pd.DataFrame(all_results)
    df_results = df_results.sort_values('return_pct', ascending=False)

    # Display top results
    print("\n" + "="*120)
    print(f"TOP {CONFIG['TOP_N_RESULTS']} BEST PERFORMING STRATEGIES")
    print("="*120)

    top_results = df_results.head(CONFIG['TOP_N_RESULTS'])

    # Format the table
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', 50)

    print(top_results[['ticker', 'strategy', 'params', 'return_pct', 'sharpe', 'max_drawdown', 'total_trades', 'win_rate']].to_string(index=False))

    print("\n" + "="*120)
    print("SUMMARY STATISTICS")
    print("="*120)
    print(f"Total Strategies Tested: {len(df_results)}")
    print(f"Profitable Strategies: {len(df_results[df_results['return_pct'] > 0])}")
    print(f"Loss-Making Strategies: {len(df_results[df_results['return_pct'] < 0])}")
    print(f"Average Return: {df_results['return_pct'].mean():.2f}%")
    print(f"Best Return: {df_results['return_pct'].max():.2f}%")
    print(f"Worst Return: {df_results['return_pct'].min():.2f}%")
    print(f"Average Sharpe Ratio: {df_results['sharpe'].mean():.2f}")
    print("="*120)

    # Save results to CSV
    df_results.to_csv('backtest_results.csv', index=False)
    print("\nFull results saved to 'backtest_results.csv'")

    return df_results


# ==================== MAIN EXECUTION ====================
if __name__ == '__main__':
    results = optimize_strategies()

NIFTY 50 STRATEGY OPTIMIZER
Configuration:
  - Period: 2023-11-06 to 2025-11-05
  - Initial Capital: ₹100,000
  - Commission: 0.1%
  - Max Strategies to Test: 10000

✓ RELIANCE.NS
✓ TCS.NS
✓ HDFCBANK.NS
✓ INFY.NS
✓ ICICIBANK.NS
✓ HINDUNILVR.NS
✓ ITC.NS
✓ SBIN.NS
✓ BHARTIARTL.NS
✓ KOTAKBANK.NS
✓ LT.NS
✓ AXISBANK.NS
✓ ASIANPAINT.NS
✓ MARUTI.NS
✓ HCLTECH.NS
✓ WIPRO.NS
✓ ULTRACEMCO.NS
✓ TITAN.NS
✓ BAJFINANCE.NS
✓ NESTLEIND.NS
✓ SUNPHARMA.NS
✓ TECHM.NS
✓ ONGC.NS
✓ NTPC.NS
✓ TATAMOTORS.NS
✓ POWERGRID.NS
✓ M&M.NS
✓ TATASTEEL.NS
✓ BAJAJFINSV.NS
✓ ADANIPORTS.NS
✓ COALINDIA.NS
✓ HINDALCO.NS
✓ DIVISLAB.NS
✓ HEROMOTOCO.NS
✓ BRITANNIA.NS
✓ CIPLA.NS
✓ EICHERMOT.NS
✓ DRREDDY.NS
✓ JSWSTEEL.NS
✓ GRASIM.NS
✓ INDUSINDBK.NS
✓ APOLLOHOSP.NS
✓ BPCL.NS
✓ TATACONSUM.NS
✓ SHRIRAMFIN.NS
✓ ADANIENT.NS
✓ SBILIFE.NS
✓ HDFCLIFE.NS
✓ BAJAJ-AUTO.NS
✓ LTIM.NS

Successfully loaded 50 stocks

Generating strategy combinations...
Progress: 100/10000 tests completed...
Progress: 200/10000 tests completed...
Progress: 300/1

In [8]:
!pip install plotly tabulate pandas seaborn matplotlib


In [9]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import matplotlib.pyplot as plt
from tabulate import tabulate

# ==================== CONFIGURATION ====================
CSV_FILE = '/content/backtest_results.csv'
TOP_N = 30  # Number of top strategies to display

# ==================== LOAD DATA ====================
print("Loading backtest results...")
df = pd.read_csv(CSV_FILE)

# Clean and prepare data
df['sharpe'] = pd.to_numeric(df['sharpe'], errors='coerce').fillna(0)
df['return_pct'] = pd.to_numeric(df['return_pct'], errors='coerce').fillna(0)
df['max_drawdown'] = pd.to_numeric(df['max_drawdown'], errors='coerce').fillna(0)
df['win_rate'] = pd.to_numeric(df['win_rate'], errors='coerce').fillna(0)
df['total_trades'] = pd.to_numeric(df['total_trades'], errors='coerce').fillna(0)

print(f"Loaded {len(df)} backtest results\n")

# ==================== BEAUTIFUL TABLE DISPLAY ====================
print("="*150)
print(f"{'TOP PERFORMING STRATEGIES':^150}")
print("="*150)

top_strategies = df.nlargest(TOP_N, 'return_pct')

# Format for display
display_df = top_strategies[['ticker', 'strategy', 'return_pct', 'sharpe', 'max_drawdown', 'total_trades', 'win_rate']].copy()
display_df.columns = ['Ticker', 'Strategy', 'Return %', 'Sharpe', 'Max DD %', 'Trades', 'Win Rate %']
display_df['Return %'] = display_df['Return %'].apply(lambda x: f"{x:.2f}")
display_df['Sharpe'] = display_df['Sharpe'].apply(lambda x: f"{x:.3f}")
display_df['Max DD %'] = display_df['Max DD %'].apply(lambda x: f"{x:.2f}")
display_df['Win Rate %'] = display_df['Win Rate %'].apply(lambda x: f"{x:.2f}")

# Print beautiful table
print(tabulate(display_df, headers='keys', tablefmt='fancy_grid', showindex=False))
print("="*150 + "\n")

# ==================== SUMMARY STATISTICS TABLE ====================
print("="*150)
print(f"{'SUMMARY STATISTICS':^150}")
print("="*150)

summary_stats = pd.DataFrame({
    'Metric': [
        'Total Strategies Tested',
        'Profitable Strategies',
        'Loss-Making Strategies',
        'Win Rate',
        'Average Return %',
        'Median Return %',
        'Best Return %',
        'Worst Return %',
        'Std Dev of Returns %',
        'Average Sharpe Ratio',
        'Average Max Drawdown %',
        'Average Trades per Strategy'
    ],
    'Value': [
        len(df),
        len(df[df['return_pct'] > 0]),
        len(df[df['return_pct'] < 0]),
        f"{(len(df[df['return_pct'] > 0]) / len(df) * 100):.2f}%",
        f"{df['return_pct'].mean():.2f}%",
        f"{df['return_pct'].median():.2f}%",
        f"{df['return_pct'].max():.2f}%",
        f"{df['return_pct'].min():.2f}%",
        f"{df['return_pct'].std():.2f}%",
        f"{df['sharpe'].mean():.3f}",
        f"{df['max_drawdown'].mean():.2f}%",
        f"{df['total_trades'].mean():.1f}"
    ]
})

print(tabulate(summary_stats, headers='keys', tablefmt='fancy_grid', showindex=False))
print("="*150 + "\n")

# ==================== STRATEGY PERFORMANCE TABLE ====================
print("="*150)
print(f"{'STRATEGY TYPE PERFORMANCE':^150}")
print("="*150)

strategy_performance = df.groupby('strategy').agg({
    'return_pct': ['mean', 'median', 'max', 'min', 'std', 'count'],
    'sharpe': 'mean',
    'win_rate': 'mean',
    'max_drawdown': 'mean'
}).round(2)

strategy_performance.columns = ['Avg Return %', 'Median Return %', 'Best Return %', 'Worst Return %', 'Std Dev %', 'Count', 'Avg Sharpe', 'Avg Win Rate %', 'Avg Max DD %']
strategy_performance = strategy_performance.sort_values('Avg Return %', ascending=False)

print(tabulate(strategy_performance, headers='keys', tablefmt='fancy_grid'))
print("="*150 + "\n")

# ==================== STOCK PERFORMANCE TABLE ====================
print("="*150)
print(f"{'TOP 20 STOCKS BY AVERAGE RETURN':^150}")
print("="*150)

stock_performance = df.groupby('ticker').agg({
    'return_pct': ['mean', 'max', 'min', 'count'],
    'sharpe': 'mean',
    'win_rate': 'mean'
}).round(2)

stock_performance.columns = ['Avg Return %', 'Best Return %', 'Worst Return %', 'Strategies Tested', 'Avg Sharpe', 'Avg Win Rate %']
stock_performance = stock_performance.sort_values('Avg Return %', ascending=False).head(20)

print(tabulate(stock_performance, headers='keys', tablefmt='fancy_grid'))
print("="*150 + "\n")

# ==================== VISUALIZATIONS ====================
print("Generating visualizations...\n")

# 1. Top 20 Strategies Bar Chart
fig1 = go.Figure()
top_20 = df.nlargest(20, 'return_pct')
fig1.add_trace(go.Bar(
    x=top_20.index,
    y=top_20['return_pct'],
    text=top_20['return_pct'].apply(lambda x: f"{x:.2f}%"),
    textposition='outside',
    marker=dict(
        color=top_20['return_pct'],
        colorscale='RdYlGn',
        showscale=True,
        colorbar=dict(title="Return %")
    ),
    hovertemplate='<b>%{customdata[0]}</b><br>Strategy: %{customdata[1]}<br>Return: %{y:.2f}%<br>Sharpe: %{customdata[2]:.3f}<br>Max DD: %{customdata[3]:.2f}%<extra></extra>',
    customdata=np.column_stack((top_20['ticker'], top_20['strategy'], top_20['sharpe'], top_20['max_drawdown']))
))
fig1.update_layout(
    title='Top 20 Strategies by Return %',
    xaxis_title='Strategy Index',
    yaxis_title='Return %',
    height=600,
    template='plotly_dark',
    showlegend=False
)
fig1.show()

# 2. Return Distribution Histogram
fig2 = go.Figure()
fig2.add_trace(go.Histogram(
    x=df['return_pct'],
    nbinsx=50,
    marker=dict(
        color='skyblue',
        line=dict(color='black', width=1)
    ),
    hovertemplate='Return Range: %{x}<br>Count: %{y}<extra></extra>'
))
fig2.add_vline(x=0, line_dash="dash", line_color="red", annotation_text="Breakeven")
fig2.add_vline(x=df['return_pct'].mean(), line_dash="dash", line_color="green", annotation_text="Mean")
fig2.update_layout(
    title='Distribution of Returns Across All Strategies',
    xaxis_title='Return %',
    yaxis_title='Frequency',
    height=500,
    template='plotly_white'
)
fig2.show()

# 3. Sharpe Ratio vs Return Scatter Plot
fig3 = px.scatter(
    df,
    x='sharpe',
    y='return_pct',
    color='strategy',
    size='total_trades',
    hover_data=['ticker', 'win_rate', 'max_drawdown'],
    title='Risk-Adjusted Returns: Sharpe Ratio vs Return %',
    labels={'sharpe': 'Sharpe Ratio', 'return_pct': 'Return %'},
    height=700
)
fig3.update_layout(template='plotly_dark')
fig3.show()

# 4. Strategy Performance Comparison Box Plot
fig4 = go.Figure()
for strategy in df['strategy'].unique():
    strategy_data = df[df['strategy'] == strategy]['return_pct']
    fig4.add_trace(go.Box(
        y=strategy_data,
        name=strategy,
        boxmean='sd'
    ))
fig4.update_layout(
    title='Return Distribution by Strategy Type',
    yaxis_title='Return %',
    xaxis_title='Strategy',
    height=600,
    template='plotly_white',
    showlegend=False
)
fig4.show()

# 5. Top Stocks Performance
top_stocks = df.groupby('ticker')['return_pct'].mean().nlargest(15)
fig5 = go.Figure()
fig5.add_trace(go.Bar(
    x=top_stocks.values,
    y=top_stocks.index,
    orientation='h',
    marker=dict(
        color=top_stocks.values,
        colorscale='Viridis',
        showscale=True
    ),
    text=top_stocks.values.round(2),
    textposition='outside',
    hovertemplate='<b>%{y}</b><br>Avg Return: %{x:.2f}%<extra></extra>'
))
fig5.update_layout(
    title='Top 15 Stocks by Average Return %',
    xaxis_title='Average Return %',
    yaxis_title='Stock Ticker',
    height=600,
    template='plotly_dark'
)
fig5.show()

# 6. Heatmap: Strategy vs Stock Performance
pivot_table = df.pivot_table(
    values='return_pct',
    index='strategy',
    columns='ticker',
    aggfunc='mean'
)
top_stocks_list = df.groupby('ticker')['return_pct'].mean().nlargest(15).index
pivot_subset = pivot_table[top_stocks_list]

fig6 = go.Figure(data=go.Heatmap(
    z=pivot_subset.values,
    x=pivot_subset.columns,
    y=pivot_subset.index,
    colorscale='RdYlGn',
    text=np.round(pivot_subset.values, 2),
    texttemplate='%{text}',
    textfont={"size": 10},
    hovertemplate='Strategy: %{y}<br>Stock: %{x}<br>Return: %{z:.2f}%<extra></extra>'
))
fig6.update_layout(
    title='Strategy Performance Heatmap (Top 15 Stocks)',
    xaxis_title='Stock Ticker',
    yaxis_title='Strategy',
    height=600,
    template='plotly_dark'
)
fig6.show()

# 7. Win Rate vs Return Scatter
fig7 = px.scatter(
    df[df['total_trades'] > 5],  # Only strategies with meaningful trade count
    x='win_rate',
    y='return_pct',
    color='strategy',
    size='total_trades',
    hover_data=['ticker', 'sharpe', 'max_drawdown'],
    title='Win Rate vs Return % (Min 5 trades)',
    labels={'win_rate': 'Win Rate %', 'return_pct': 'Return %'},
    height=700
)
fig7.update_layout(template='plotly_white')
fig7.show()

# 8. Drawdown vs Return Scatter
fig8 = px.scatter(
    df,
    x='max_drawdown',
    y='return_pct',
    color='sharpe',
    size='total_trades',
    hover_data=['ticker', 'strategy', 'win_rate'],
    title='Risk Analysis: Max Drawdown vs Return %',
    labels={'max_drawdown': 'Max Drawdown %', 'return_pct': 'Return %'},
    height=700,
    color_continuous_scale='RdYlGn'
)
fig8.update_layout(template='plotly_dark')
fig8.show()

# 9. Strategy Count Pie Chart
strategy_counts = df['strategy'].value_counts()
fig9 = go.Figure(data=[go.Pie(
    labels=strategy_counts.index,
    values=strategy_counts.values,
    hole=0.4,
    textinfo='label+percent',
    hovertemplate='<b>%{label}</b><br>Tests: %{value}<br>Percentage: %{percent}<extra></extra>'
)])
fig9.update_layout(
    title='Distribution of Tested Strategies',
    height=600,
    template='plotly_dark'
)
fig9.show()

# 10. Performance Metrics Comparison
top_10 = df.nlargest(10, 'return_pct')
fig10 = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Return %', 'Sharpe Ratio', 'Max Drawdown %', 'Win Rate %'),
    specs=[[{'type': 'bar'}, {'type': 'bar'}],
           [{'type': 'bar'}, {'type': 'bar'}]]
)

# Return %
fig10.add_trace(go.Bar(
    x=top_10.index,
    y=top_10['return_pct'],
    name='Return %',
    marker_color='lightgreen',
    text=top_10['return_pct'].round(2),
    textposition='outside'
), row=1, col=1)

# Sharpe Ratio
fig10.add_trace(go.Bar(
    x=top_10.index,
    y=top_10['sharpe'],
    name='Sharpe',
    marker_color='lightblue',
    text=top_10['sharpe'].round(2),
    textposition='outside'
), row=1, col=2)

# Max Drawdown
fig10.add_trace(go.Bar(
    x=top_10.index,
    y=top_10['max_drawdown'],
    name='Max DD %',
    marker_color='salmon',
    text=top_10['max_drawdown'].round(2),
    textposition='outside'
), row=2, col=1)

# Win Rate
fig10.add_trace(go.Bar(
    x=top_10.index,
    y=top_10['win_rate'],
    name='Win Rate %',
    marker_color='gold',
    text=top_10['win_rate'].round(2),
    textposition='outside'
), row=2, col=2)

fig10.update_layout(
    title_text='Top 10 Strategies - Key Metrics Comparison',
    showlegend=False,
    height=800,
    template='plotly_white'
)
fig10.show()

print("\n" + "="*150)
print("VISUALIZATION COMPLETE!")
print("="*150)
print("\nAll charts displayed above. Scroll up to see:")
print("  1. Top 20 Strategies Bar Chart")
print("  2. Return Distribution Histogram")
print("  3. Sharpe Ratio vs Return Scatter")
print("  4. Strategy Performance Box Plot")
print("  5. Top Stocks Performance")
print("  6. Strategy vs Stock Heatmap")
print("  7. Win Rate vs Return Analysis")
print("  8. Risk Analysis (Drawdown vs Return)")
print("  9. Strategy Distribution Pie Chart")
print(" 10. Top 10 Metrics Comparison")
print("="*150)

Loading backtest results...
Loaded 6372 backtest results

                                                              TOP PERFORMING STRATEGIES                                                               
╒═══════════════╤═══════════════════════╤════════════╤══════════╤════════════╤══════════╤══════════════╕
│ Ticker        │ Strategy              │   Return % │   Sharpe │   Max DD % │   Trades │   Win Rate % │
╞═══════════════╪═══════════════════════╪════════════╪══════════╪════════════╪══════════╪══════════════╡
│ MARUTI.NS     │ RSIStrategy           │       5.57 │   -1.99  │       1.01 │        3 │       100    │
├───────────────┼───────────────────────┼────────────┼──────────┼────────────┼──────────┼──────────────┤
│ ULTRACEMCO.NS │ StochasticStrategy    │       4.66 │   -3.847 │       0.83 │        6 │       100    │
├───────────────┼───────────────────────┼────────────┼──────────┼────────────┼──────────┼──────────────┤
│ MARUTI.NS     │ MACDStrategy          │       4.62 │  


VISUALIZATION COMPLETE!

All charts displayed above. Scroll up to see:
  1. Top 20 Strategies Bar Chart
  2. Return Distribution Histogram
  3. Sharpe Ratio vs Return Scatter
  4. Strategy Performance Box Plot
  5. Top Stocks Performance
  6. Strategy vs Stock Heatmap
  7. Win Rate vs Return Analysis
  8. Risk Analysis (Drawdown vs Return)
  9. Strategy Distribution Pie Chart
 10. Top 10 Metrics Comparison
